In [5]:
import pandas as pd
import csv
import os

with open(
    "/home/s17gmikh/FCD-Detection/meld_graph/data/preprocessed/MELD_BONN_dataset_full.csv",
    "r",
) as f:
    data = pd.read_csv(
        f,
        sep=",",
        engine="python",  # нужен движок Python, чтобы поддерживался escapechar
        quoting=csv.QUOTE_NONE,  # не искать кавычки
        escapechar="\\",  # '\' перед ',' будет означать «не разделитель»
    )

# 2) вытаскиваем sub-ID

data["subject_id"] = data["DATA_PATH"].apply(
    lambda p: (
        os.path.basename(p).split("_patient")[0].split("_control")[0]
        if isinstance(p, str)
        else None
    )
)
print(data["subject_id"])

0       MELD_H2_15T_FCD_0001
1       MELD_H2_15T_FCD_0002
2       MELD_H2_15T_FCD_0003
3       MELD_H2_15T_FCD_0005
4       MELD_H2_15T_FCD_0006
                ...         
1125     MELD_H16_3T_FCD_009
1126     MELD_H16_3T_FCD_010
1127     MELD_H16_3T_FCD_011
1128     MELD_H16_3T_FCD_012
1129     MELD_H16_3T_FCD_013
Name: subject_id, Length: 1130, dtype: object


In [1]:
import torch
import torch.nn as nn

# "сырые" логиты модели
logits = torch.tensor(
    [[1.2, -0.3, 0.1], [-0.5, 0.7, 2.0]], dtype=torch.float32
)  # [B, C]
targets = torch.tensor([0, 2])  # правильные классы

# Вариант 1: CrossEntropy на логитах
ce = nn.CrossEntropyLoss(reduction="none")
loss_ce = ce(logits, targets)

# Вариант 2: LogSoftmax + NLLLoss
log_probs = torch.log_softmax(logits, dim=1)  # сами считаем LogSoftmax
nll = nn.NLLLoss(reduction="none")
loss_nll = nll(log_probs, targets)

print("loss_ce :", loss_ce)  # одинаково поэлементно
print("loss_nll:", loss_nll)
print("max |diff| =", (loss_ce - loss_nll).abs().max().item())

loss_ce : tensor([0.4421, 0.3035])
loss_nll: tensor([0.4421, 0.3035])
max |diff| = 0.0


In [2]:
import pandas as pd

# путь к твоему csv
csv_path = "/home/s17gmikh/FCD-Detection/meld_graph/data/preprocessed/all_augmented_reports.csv"

# читаем csv
df = pd.read_csv(csv_path)

# было строк
n_before = len(df)

# удаляем дубликаты по колонке subject_id (оставляем первую встречу)
df = df.drop_duplicates(subset=["subject_id"], keep="first")

# стало строк
n_after = len(df)

# сохраняем обратно
df.to_csv(csv_path, index=False)

print(
    f"✅ Удалено {n_before - n_after} дубликатов, осталось {n_after} уникальных subject_id."
)

✅ Удалено 617 дубликатов, осталось 1129 уникальных subject_id.
